### 1. Veri Setini Yükleme ve Keşif

Veri setimiz `sms_spam_train.csv` dosyasından yüklenecek. Dosyada her satırda bir SMS mesajı ve mesajın "ham" veya "spam" olarak etiketlendiği iki sütun (Message, Label) bulunuyor.



In [2]:
import pandas as pd

# CSV dosyasını yükleyin
df = pd.read_csv("sms_spam_train.csv")
df.head()


C:\Users\Hakan\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,Message,Label
0,Whats upp ac enisteee: -) senin memlekette bul...,ham
1,"Kampanya, VakifBank Worldcardla 31 Mayisa kada...",spam
2,I wont. So wat's wit the guys,ham
3,Bu arada son dakika evi düşürdük,ham
4,No that just means you have a fat head,ham


In [3]:
df.info()
print(df['Label'].value_counts())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Message  10000 non-null  object
 1   Label    10000 non-null  object
dtypes: object(2)
memory usage: 156.4+ KB
Label
ham     6621
spam    3379
Name: count, dtype: int64


### 2. Eğitim ve Validasyon Verilerinin Oluşturulması

Sağlanan veriyi %80 eğitim ve %20 validasyon olacak şekilde böleceğiz. Ayrıca, etiket dengesini korumak için stratified split kullanacağız.


In [4]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Label'])

print("Eğitim verisi örnek sayısı:", len(train_df))
print("Validasyon verisi örnek sayısı:", len(val_df))


Eğitim verisi örnek sayısı: 8000
Validasyon verisi örnek sayısı: 2000


### 3. Veri Setini Prompt Formatına Dönüştürme

Mistral modelini instruction fine-tuning ile eğiteceğimiz için, her örneği aşağıdaki formatta hazırlayacağız:



In [5]:
def format_prompt(row):
    """
    Her bir örnek için prompt formatını oluşturur.
    """
    prompt = (
        f"Aşağıdaki SMS mesajını spam mı yoksa ham mı olarak sınıflandırınız?\n"
        f"Mesaj: {row['Message']}\n"
        f"Cevap: {row['Label']}"
    )
    return prompt

# Eğitim ve validasyon verileri için prompt'ları oluşturun
train_prompts = train_df.apply(format_prompt, axis=1).tolist()
val_prompts = val_df.apply(format_prompt, axis=1).tolist()

# Örnek bir prompt kontrolü
print(train_prompts[0])


Aşağıdaki SMS mesajını spam mı yoksa ham mı olarak sınıflandırınız?
Mesaj: I don't run away frm u... I walk slowly &amp; it kills me that u don't care enough to stop me...
Cevap: ham


Şimdi, Hugging Face `datasets` kütüphanesiyle kullanılacak formata (örneğin, bir sözlük içinde "text" anahtarı) dönüştürelim.


In [6]:
from datasets import Dataset

train_dataset = Dataset.from_dict({"text": train_prompts})
val_dataset = Dataset.from_dict({"text": val_prompts})

# İlk örneği kontrol edelim
print(train_dataset[0])


{'text': "Aşağıdaki SMS mesajını spam mı yoksa ham mı olarak sınıflandırınız?\nMesaj: I don't run away frm u... I walk slowly &amp; it kills me that u don't care enough to stop me...\nCevap: ham"}


### 4. Model ve Tokenizer’ın Kurulumu

Mistral-7B-Instruct modelini Hugging Face üzerinden indirip, 4-bit quantization ve gradient checkpointing ile model belleğini optimize ediyoruz.


In [6]:
import os
os.environ["TRANSFORMERS_OFFLINE"] = "1"


In [7]:
local_model_path = "mistral-model"
tokenizer = AutoTokenizer.from_pretrained(local_model_path)
model = AutoModelForCausalLM.from_pretrained(local_model_path)


NameError: name 'AutoTokenizer' is not defined

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from peft import prepare_model_for_kbit_training  # PEFT içindeki yardımcı fonksiyonu kullanıyoruz

model_id = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_id)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Modeli 4-bit quantization ile yükleyin
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)


`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

C:\Users\Hakan\anaconda3\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Hakan\.cache\huggingface\hub\models--mistralai--Mistral-7B-Instruct-v0.3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model-00001-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]